# Lab 3: Building Agents with memory

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

## Section 0: Setup a Letta client

In [ ]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")
# client = Letta(token="LETTA_API_KEY")

In [ ]:
def print_message(message):
    if message.message_type == "reasoning_message":
        print("🧠 Reasoning: " + message.reasoning)
    elif message.message_type == "assistant_message":
        print("🤖 Agent: " + message.content)
    elif message.message_type == "tool_call_message":
        print("🔧 Tool Call: " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message":
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message":
        print("👤 User Message: " + message.content)

## Section 1: Creating a simple agent with memory

### Creating an agent

In [ ]:
agent_state = client.agents.create(
    name="simple_agent",
    memory_blocks=[
        {
          "label": "human",
          "value": "My name is Charles",
          "limit": 10000 # character limit
        },
        {
          "label": "persona",
          "value": "You are a helpful assistant and you always use emojis"
        }
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-3-small"
)

### Messaging an agent

In [ ]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "hows it going????"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

### Viewing usage information

In [ ]:
# if we want to print the usage stats
print(response.usage.completion_tokens)
print(response.usage.prompt_tokens)
print(response.usage.step_count)

### Understanding agent state

In [ ]:
print(agent_state.system)

In [ ]:
[t.name for t in agent_state.tools]

📝 Note: Memory blocks are returned as an unordered list and you may receive blocks in an order different than in the video

In [ ]:
agent_state.memory

📝 Note: Returned messages may differ slightly from those in the video

In [ ]:
for message in client.agents.messages.list(agent_id=agent_state.id):
    print_message(message)

In [ ]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
passages

## Section 2: Understanding core memory

### Giving agent new information

In [ ]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "my name actually Sarah "
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

In [ ]:
print(response.usage.completion_tokens)
print(response.usage.prompt_tokens)
print(response.usage.step_count)

### Retrieving new values

In [ ]:
client.agents.blocks.retrieve(
    agent_id=agent_state.id,
    block_label="human"
).value

## Section 3: Understanding archival memory

### Saving information to archival memory

In [ ]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
passages

In [ ]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Save the information that 'bob loves cats' to archival"
        }
    ]
)

# if we want to print the messages
for message in response.messages:
    print_message(message)

In [ ]:
passages = client.agents.passages.list(
    agent_id=agent_state.id,
)
[passage.text for passage in passages]

### Explicitly creating archival memories

In [ ]:
client.agents.passages.create(
    agent_id=agent_state.id,
    text="Bob's loves boston terriers",
)

In [ ]:
# send a message to the agent
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "What animals do I like? Search archival."
        }
    ]
)

for message in response.messages:
    print_message(message)